In [1]:
# ==============================================================================
# PASO 1: CONFIGURACIÓN INICIAL Y CORRECCIÓN DE DIRECTORIOS
# ==============================================================================
import os
import nltk

# Clona el repositorio desde GitHub.
!git clone https://github.com/willmedina/aispam.git
%cd aispam

# Renombra los directorios para que sean válidos en Python.
print("\n⏳ Renombrando directorios...")
os.rename("0_DATASETS", "datasets")
os.rename("1_GENERACION", "generacion")
os.rename("2_PRE_PROCESAMIENTO", "pre_procesamiento")
os.rename("3_ENTRENAMIENTO", "entrenamiento")
os.rename("4_DESPLIEGUE", "despliegue")
print("✅ Directorios renombrados.")

# Instala las librerías necesarias.
print("\n⏳ Instalando dependencias...")
!pip install -q pytest pytest-cov mock flask tensorflow pandas numpy scikit-learn markovify
print("✅ Dependencias instaladas.")

# Descarga los recursos 'stopwords' y 'punkt' de NLTK.
print("\n⏳ Descargando recursos de NLTK...")
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
print("✅ Recursos de NLTK descargados.")

# Crea archivos __init__.py.
print("\n⏳ Creando archivos __init__.py...")
project_dirs = [
    ".", "generacion", "pre_procesamiento", "entrenamiento",
    "despliegue", "despliegue/API"
]
for directory in project_dirs:
    if os.path.isdir(directory):
        init_path = os.path.join(directory, "__init__.py")
        !touch {init_path}

# Crea el directorio de pruebas.
!mkdir -p tests
!touch tests/__init__.py

print("\n✅ Entorno listo y corregido.")

Cloning into 'aispam'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 131 (delta 1), reused 4 (delta 1), pack-reused 122 (from 1)
Receiving objects: 100% (131/131), 35.35 MiB | 8.14 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Updating files: 100% (45/45), done.
/content/aispam

⏳ Renombrando directorios...
✅ Directorios renombrados.

⏳ Instalando dependencias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 17.3 MB/s eta 0:00:00
✅ Dependencias instaladas.

⏳ Descargando recursos de NLTK...
✅ Recursos de NLTK descargados.

⏳ Creando archivos __init__.py...

✅ Entorno listo y corregido.


In [2]:
%%writefile tests/test_generacion.py
import pytest
import pandas as pd
from unittest.mock import patch, MagicMock

# Se importa directamente desde el módulo renombrado.
from aispam.generacion import augmeter_markov, generador_spam_organico2

# --- Pruebas para augmeter_markov.py ---
# PRUEBA QUE YA PASA (NO SE TOCA)
def test_clean_text_for_final_output():
    texto_sucio = '“HOLA, MUNDO!”\nEsto es una prueba, con http://url.com y emojis 👍.'
    texto_esperado = "'hola mundo!' esto es una prueba con http://url.com y emojis ."
    assert augmeter_markov.clean_text_for_final_output(texto_sucio) == texto_esperado

# PRUEBA CORREGIDA
def test_augmeter_markov_logic():
    # SOLUCIÓN: Se simula ('mock') la función make_sentence de markovify para
    # que la prueba no dependa de su aleatoriedad.
    with patch('markovify.Text.make_sentence', return_value="Frase generada por el mock.") as mock_make_sentence:
        input_text = ["Un texto de ejemplo simple es suficiente ahora."]
        markov_model = augmeter_markov.train_markov_model(input_text, state_size=1)
        assert markov_model is not None

        generated = augmeter_markov.generate_augmented_messages(markov_model, num_messages=1, label='spam')

        # Se verifica que nuestra función haya llamado al método simulado.
        mock_make_sentence.assert_called_once()
        # Se verifica que nuestra función haya procesado la salida del mock.
        assert len(generated) == 1
        assert generated[0]['mensaje'] == "Frase generada por el mock."

# --- Pruebas para generador_spam_organico2.py ---
# PRUEBAS QUE YA PASAN (NO SE TOCAN)
@pytest.mark.parametrize("function_name", ["generate_spam_message", "generate_ham_message"])
@patch('aispam.generacion.generador_spam_organico2.generate_text_robust')
def test_generate_message_functions(mock_generate_text, function_name):
    mock_generate_text.return_value = "texto generado"
    func_to_test = getattr(generador_spam_organico2, function_name)
    mensaje, tipo = func_to_test()
    assert mensaje == "texto generado"

@patch('aispam.generacion.generador_spam_organico2.generate_spam_message')
@patch('aispam.generacion.generador_spam_organico2.generate_ham_message')
def test_create_dataset_function(mock_gen_ham, mock_gen_spam):
    mock_gen_spam.return_value = ("spam message", "spam")
    mock_gen_ham.return_value = ("ham message", "ham")
    df = generador_spam_organico2.create_dataset(total=10, spam_ratio=0.5, max_workers=2)
    assert isinstance(df, pd.DataFrame)
    assert len(df) == 10
    assert df['tipo'].value_counts()['spam'] == 5

print("✅ Archivo 'tests/test_generacion.py' final y corregido.")

Writing tests/test_generacion.py


In [3]:
%%writefile tests/test_pre_procesamiento.py
import pytest
import pandas as pd
from unittest.mock import patch, MagicMock

import matplotlib
matplotlib.use('Agg')

# Se prueba el flujo completo del script, ya que las funciones se llaman secuencialmente.
@patch('pandas.read_csv')
@patch('joblib.dump')
def test_pre_procesamiento_main_flow(mock_joblib_dump, mock_read_csv):
    # SOLUCIÓN: Se crea un DataFrame con texto que no será eliminado por completo
    # por el filtro de stopwords, evitando el error de "empty vocabulary".
    # También tiene la columna 'mensaje' en minúsculas y 20 filas para .sample(20).
    mock_data = {
        'mensaje': [f'correo importante sobre reunion {i}' for i in range(20)],
        'tipo': ['ham', 'spam'] * 10
    }
    mock_read_csv.return_value = pd.DataFrame(mock_data)

    # La importación DENTRO de la prueba asegura que los mocks estén activos.
    try:
        from aispam.pre_procesamiento import pre_procesamiento
        # Si la importación no falla, la prueba pasa.
        assert True
    except Exception as e:
        pytest.fail(f"La importación del script de pre-procesamiento falló inesperadamente: {e}")

print("✅ Archivo 'tests/test_pre_procesamiento.py' final y corregido.")

Writing tests/test_pre_procesamiento.py


In [4]:
%%writefile tests/test_entrenamiento.py
import pytest
import pandas as pd
import numpy as np
from unittest.mock import MagicMock, patch

import matplotlib
matplotlib.use('Agg')

@patch('pandas.read_csv')
@patch('tensorflow.keras.models.Sequential.fit')
@patch('tensorflow.keras.models.Sequential.predict')
@patch('tensorflow.keras.models.Sequential.evaluate') # Este es el mock que vamos a ajustar
@patch('tensorflow.keras.models.save_model')
def test_model_architecture_and_script_flow(mock_save, mock_eval, mock_predict, mock_fit, mock_read_csv):
    mock_read_csv.return_value = pd.DataFrame({
        'processed_Text': ['texto bueno'] * 10,
        'tipo_n': [0] * 5 + [1] * 5
    })

    mock_history = MagicMock()
    mock_history.history = {
        'loss': [1.0], 'val_loss': [1.0],
        'accuracy': [0.5], 'val_accuracy': [0.5]
    }
    mock_fit.return_value = mock_history

    mock_predict.return_value = np.array([[0.1], [0.9]])

    # SOLUCIÓN: Se configura el mock de evaluate para que devuelva una lista de 4 floats,
    # que es lo que el código espera al desempaquetar.
    mock_eval.return_value = [0.5, 0.9, 0.85, 0.92] # Simula loss, accuracy, precision, recall

    # Se importa el script DENTRO de la prueba.
    from aispam.entrenamiento import entrenamiento

    assert isinstance(entrenamiento.model, entrenamiento.Sequential)
    mock_fit.assert_called_once()
    mock_save.assert_called_once()

print("✅ Archivo 'tests/test_entrenamiento.py' final y corregido.")

Writing tests/test_entrenamiento.py


In [5]:
%%writefile tests/test_app.py
import pytest
import numpy as np
from unittest.mock import patch, MagicMock

import matplotlib
matplotlib.use('Agg')

# SOLUCIÓN: Se parchean las funciones específicas de carga, no 'builtins.open'.
@patch('joblib.load')
@patch('tensorflow.keras.models.load_model')
def test_api_endpoints(mock_load_model, mock_joblib_load):
    # Se simula la carga exitosa de los modelos.
    mock_model_instance = MagicMock()
    mock_vectorizer_instance = MagicMock()
    mock_load_model.return_value = mock_model_instance
    mock_joblib_load.return_value = mock_vectorizer_instance

    # Se simula el método 'transform' que la app necesita del vectorizador.
    mock_vectorizer_instance.transform.return_value = MagicMock()

    from aispam.despliegue.API.app import app

    app.config['TESTING'] = True
    client = app.test_client()

    # Se simula el método de predicción del modelo para esta prueba.
    with patch('aispam.despliegue.API.app.model.predict', return_value=np.array([[0.95]])) as mock_predict:
        response = client.post('/predict', json={"message": "gana un premio ahora"})
        assert response.status_code == 200
        assert response.get_json()['prediction_label'] == 'spam'

    # Se prueba el endpoint de importación de CSV.
    csv_payload = [{"metadata": {}, "registros": [{"mensaje": "texto1"}]}]
    with patch('aispam.despliegue.API.app.model.predict', return_value=np.array([[0.10]])) as mock_predict_csv:
        response_csv = client.post('/importcsv', json=csv_payload)
        assert response_csv.status_code == 200
        assert 'probabilidad_spam' in response_csv.get_json()[0]['registros'][0]


print("✅ Archivo 'tests/test_app.py' final y corregido.")

Writing tests/test_app.py


In [6]:
# ==============================================================================
# PASO FINAL: EJECUTAR TODAS LAS PRUEBAS UNITARIAS
# ==============================================================================
print("🚀 Ejecutando todo el conjunto de pruebas de Pytest...")

# El comando ejecutará todas las pruebas en el directorio 'tests'.
# Con todas las correcciones, la mayoría o todas deberían pasar.
!python -m pytest tests/ -v

print("\n🎉 ¡Plan de pruebas unitarias completado!")

🚀 Ejecutando todo el conjunto de pruebas de Pytest...
============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/aispam
plugins: cov-6.2.1, langsmith-0.3.45, typeguard-4.4.3, anyio-4.9.0
collected 8 items                                                              

tests/test_app.py::test_api_endpoints PASSED                             [ 12%]
tests/test_entrenamiento.py::test_model_architecture_and_script_flow PASSED [ 25%]
tests/test_generacion.py::test_clean_text_for_final_output PASSED        [ 37%]
tests/test_generacion.py::test_augmeter_markov_logic PASSED              [ 50%]
tests/test_generacion.py::test_generate_message_functions[generate_spam_message] PASSED [ 62%]
tests/test_generacion.py::test_generate_message_functions[generate_ham_message] PASSED [ 75%]
tests/test_generacion.py::test_create_dataset_function PASSED            [ 